In [67]:
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy
import pandas as pd
from IPython.display import FileLink
import geopandas as gpd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import paramiko
from io import StringIO
from shapely.geometry import MultiPoint, MultiPolygon
from shapely.ops import unary_union
import calendar
from datetime import datetime

In [68]:
def get_df_from_sql(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [69]:
# Usage with the actual path to the private key
SSH_required = 'Yes'
key_path = '/Users/rajatsansaniwal/Documents/tunnel-ssh .cer'


In [70]:
query = """
with base_fresh as (
    select
        unnest(ofd_fresh_pendency) as awb
    ,   shipping_city
    ,   last_mile_hub
    ,   cluster
    ,   slot
    ,   'Fresh' as type
    from ofd_table
    where date = (now() + interval'5.5 hours')::date
)
,
base_backlog as (
    select
        unnest(ofd_backlog_pendency) as awb
    ,   shipping_city
    ,   last_mile_hub
    ,   cluster
    ,   slot
    ,   'Backlog' as type
    from ofd_table
    where date = (now() + interval'5.5 hours')::date
)

select
    a.shipping_city
,   a.last_mile_hub
,   a.cluster
,   o.shipping_pincode
,   count(a.awb) as orders
from (select * 
        from base_fresh
        
        union all
        
        select *
        from base_backlog
) as a
left join ops_main o on a.awb = o.awb
where o.first_ofd_time is null
and slot = 'Evening'
group by 1, 2, 3, 4
order by 1, 2, 3, 4
"""

In [71]:
# Retrieve data into a DataFrame
df = get_df_from_sql(SSH_required, query, key_path)

print(df)

    shipping_city last_mile_hub  cluster  shipping_pincode  orders
0       Bangalore          BLDR  BLDR-07            560102      54
1       Bangalore          BLDR  BLDR-07            560103      73
2       Bangalore          BLDR  WHTF-05            560066      58
3       Bangalore          CMRJ  CMRJ-13            560002       3
4       Bangalore          CMRJ  CMRJ-13            560004       7
..            ...           ...      ...               ...     ...
308          Pune          KLWD  PSHN-11            411021       3
309          Pune          KLWD  SVNR-12            411004      30
310          Pune          KLWD  SVNR-12            411005       7
311          Pune          KLWD  SVNR-12            411016      19
312          Pune          KLWD  TLWD-13            411062       2

[313 rows x 5 columns]


In [72]:
# Reading GeoJSON
gdf_geojson = gpd.read_file("/Users/rajatsansaniwal/git_geoboards/geoboards/India_Pincodes/india_pincodes.shp")


# If the current CRS is geographic, re-project to UTM (EPSG:32644)
if gdf_geojson.crs.is_geographic:
    gdf_geojson = gdf_geojson.to_crs('EPSG:32644')



In [73]:
# print(gdf_geojson)

In [74]:
# Calculating centroids of pincodes
gdf_geojson = gdf_geojson.to_crs(epsg=4326)

gdf_geojson['latitude'] = gdf_geojson['geometry'].centroid.y
gdf_geojson['longitude'] = gdf_geojson['geometry'].centroid.x

# print(gdf_geojson.head())

In [75]:
# Ensure the pincode column datatype is consistent
df['shipping_pincode'] = df['shipping_pincode'].astype(str)
gdf_geojson['pincode'] = gdf_geojson['pincode'].astype(str)

# Merge GeoDataFrame and DataFrame
merged_gdf = gdf_geojson.merge(df, left_on='pincode', right_on='shipping_pincode')

In [76]:
dff = merged_gdf[(df["shipping_city"] == 'Bangalore')]
print(dff)

   pincode        state   district                 officename officetype  \
0   411020  Maharashtra       Pune    Botanical Garden (Pune)        S.O   
1   400054  Maharashtra     Mumbai            Santacruz(West)        S.O   
2   400101  Maharashtra     Mumbai             Kandivali East        S.O   
3   400604  Maharashtra      Thane                 Wagle I.E.        S.O   
4   400606  Maharashtra      Thane                   Jekegram        S.O   
..     ...          ...        ...                        ...        ...   
80  560093    Karnataka  Bangalore            C.V.Raman Nagar        S.O   
81  560094    Karnataka  Bangalore  R.M.V. Extension II Stage        S.O   
82  560095    Karnataka  Bangalore          Koramangala VI Bk        S.O   
83  560096    Karnataka  Bangalore              Nandinilayout        S.O   
84  560097    Karnataka  Bangalore             Vidyaranyapura        S.O   

                                             geometry   latitude  longitude  \
0   POLY

In [80]:
# Initialize Dash app
app = Dash(__name__)

# Define app layout
app.layout = html.Div([
    html.H1("Pending Fresh Orders Today", style={'text-align': 'center'}),
    dcc.Input(
        id='search_shippingcity',
        type = 'text',
        placeholder='Search Shipping City here',
        value='Bangalore',
        style={'width': "40%"}
    ),
    dcc.Input(
        id='search_last_mile_hub',
        type = 'text',
        placeholder='Search Last Mile Hub here',
        value='MRTH',
        style={'width': "40%"}
    ),
    html.Div(id='output_container_city', children=[]),
    html.Div(id='output_container_last_mile_hub', children=[]),
    html.Br(),
    html.Div([
        dcc.Graph(id='map', figure={}),
    ], style={'width': '100%', 'margin': '0 auto'})
])

# Callback to update the graph
@app.callback(
    [
        Output(component_id='output_container_city', component_property='children')
    ,   Output(component_id='output_container_last_mile_hub', component_property='children')
    ,   Output(component_id='map', component_property='figure')
    ],
    [
        Input(component_id='search_shippingcity', component_property='value')
    ,   Input(component_id='search_last_mile_hub', component_property='value' )
     ]
)
def update_graph(city_slctd, hub_slctd):
    if not city_slctd or city_slctd not in merged_gdf['shipping_city'].unique():
        return "", "", {}
    output_container_city = "Results are shown for the Shipping City: {}".format(city_slctd)
    output_container_last_mile_hub = f"Results are shown for Cluster: {hub_slctd}"
    dff = merged_gdf[(merged_gdf["shipping_city"] == city_slctd)]

    if hub_slctd:
        dff = merged_gdf[merged_gdf["last_mile_hub"] == hub_slctd]

    # Generate centroid
    centroid = dff.geometry.centroid
    centroid_wgs84 = centroid.to_crs(epsg=4326)
    dff['geometry'] = dff['geometry'].simplify(tolerance=0.001)
    
    fig = px.choropleth_mapbox(
        dff,  # Use filtered data from merged_gdf
        geojson=dff.geometry,  # Pass geometry from filtered data
        locations=dff.index,  # Use index from filtered data
        color='orders',
        hover_data=['last_mile_hub', 'cluster', 'pincode'],
        mapbox_style="carto-positron",
        center={"lat": centroid_wgs84.y.mean(), "lon": centroid_wgs84.x.mean()},
        zoom=8.5,
        opacity=0.9,
        template='plotly_dark',
        color_continuous_scale="RdYlGn_r"
    )
    fig.update_traces(
        hovertemplate='<b>Pincode:</b> %{customdata[0]}<br><b>Hub:</b> %{customdata[1]}<br><b>orders:</b> %{customdata[2]}',
        customdata=dff[['pincode', 'last_mile_hub', 'orders']],
        text=dff['pincode']
    )
    return output_container_city, output_container_last_mile_hub, fig

if __name__ == '__main__':
    app.run(debug=True, port=8054, mode='external')
#     app.run()
# print(grouped_dff[grouped_dff['shipping_city'] == 'Bangalore'])